In [1]:
import findspark
findspark.init()

from pyspark.mllib.recommendation import *
import random
from operator import *
from collections import defaultdict

In [10]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession

spark = SparkContext(master="local[2]",appName="recomendation")


In [16]:
artistData = spark.textFile(r"C:\Users\gonza\Repositorios\TestGit\Music-Recommender-System-Pyspark\artist_data_small.txt") \
    .map(lambda s:(int(s.split("\t")[0]),s.split("\t")[1]))   
    
artistAlias = spark.textFile(r"C:\Users\gonza\Repositorios\TestGit\Music-Recommender-System-Pyspark\artist_alias_small.txt")
userArtistData = spark.textFile(r"C:\Users\gonza\Repositorios\TestGit\Music-Recommender-System-Pyspark\user_artist_data_small.txt")


In [17]:
#Find the three users with the highest number of total play counts (sum of all counters) and print the user ID, the total play count, and the mean play count (average number of times a user played an artist)
# split
userArtistData = userArtistData.map(lambda s:(int(s.split(" ")[0]),int(s.split(" ")[1]),int(s.split(" ")[2])))
# create a dict
artistAliasDict = {}
dataValue = artistAlias.map(lambda s:(int(s.split("\t")[0]),int(s.split("\t")[1])))
for temp in dataValue.collect():
    artistAliasDict[temp[0]] = temp[1]

# If artistid exists, replace with artistsid from artistAlias, else retain original
userArtistData = userArtistData.map(lambda x:(x[0],artistAliasDict[x[1]] if x[1] in artistAliasDict else x[1],x[2]))

In [19]:
#Create an RDD consisting of 'userid' and 'playcount' objects of original tuple
userSum = userArtistData.map(lambda x:(x[0],x[2]))
playCount1 = userSum.map(lambda x:(x[0],x[1])).reduceByKey(lambda a,b:a+b)
playCount2 = userSum.map(lambda x:(x[0],1)).reduceByKey(lambda a,b:a+b)
playSumAndCount = playCount1.leftOuterJoin(playCount2)

In [22]:
type(playCount2)

pyspark.rdd.PipelinedRDD